# <center> Task 4: Credit Card Default Prediction </center>

**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
filepath="C:/Users/uttam/anaconda3/Technocolabs/MinorProj2/Datasets/cleaned_data.csv"
df= pd.read_csv(filepath)
df.head()

,ID,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [4]:
# Define the sigmoid function
def sigmoid(x):
    return 1 /( np.exp(-x) + 1 )


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [5]:
# Create a train/test split
# I renamed default payment next month to DEFAULT for easier readablity
X_train, X_test, y_train, y_test = train_test_split(
                            df[['PAY_1', 'LIMIT_BAL']].values, df['DEFAULT'].values,
                            test_size=0.2, random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [6]:
from sklearn.linear_model import LogisticRegression
LogisticRegressor= LogisticRegression(solver='liblinear')
LogisticRegressor

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [7]:
# Fit the logistic regression model on training data
LogisticRegressor.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
# Make predictions using `.predict()`
y_predict = LogisticRegressor.predict(X_test)

In [9]:
# Find class probabilities using `.predict_proba()`
y_predict_proba = LogisticRegressor.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [10]:
print( LogisticRegressor.intercept_,LogisticRegressor.coef_)

[-6.57647457e-11] [[ 8.27451187e-11 -6.80876727e-06]]


In [11]:
# Add column of 1s to features
one_array = np.ones((X_test.shape[0],1))
ones_with_feature = np.hstack([one_array, X_test])
ones_with_feature

array([[ 1.0e+00,  2.0e+00,  1.6e+05],
       [ 1.0e+00,  1.0e+00,  5.0e+04],
       [ 1.0e+00, -1.0e+00,  2.0e+05],
       ...,
       [ 1.0e+00, -1.0e+00,  5.0e+04],
       [ 1.0e+00,  1.0e+00,  2.3e+05],
       [ 1.0e+00,  2.0e+00,  1.0e+05]])

In [12]:
# Get coefficients and intercepts from trained model
coefficients_and_intercept = np.concatenate([LogisticRegressor.intercept_.reshape(1,1), LogisticRegressor.coef_], axis=1)
coefficients_and_intercept

array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [13]:
# Manually calculate predicted probabilities
X_linear = np.dot(coefficients_and_intercept, np.transpose(ones_with_feature))
# For checking: X_linear.shape
y_predict_prob_manual = sigmoid(X_linear)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [14]:
# Manually calculate predicted classes
y_predict_manual = (y_predict_prob_manual >= 0.5)

In [15]:
#Shapes of both predicted y values
print("Sklearn Predicted Shape: ",y_predict.shape)
print("Manual Predicted Shape: ",y_predict_manual.shape)

Sklearn Predicted Shape:  (5333,)
Manual Predicted Shape:  (1, 5333)


In [16]:
# Compare to scikit-learn's predicted classes
np.array_equal(y_predict.reshape(1,-1), y_predict_manual)

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [17]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_predict_proba[:,1]))

0.627207450280691


In [18]:
# Use manually calculated predicted probabilities to calculate ROC AUC
print(roc_auc_score(y_test, y_predict_prob_manual.reshape(y_predict_prob_manual.shape[1],)))

0.627207450280691


##### Both have same ROC AUC Score, as expected

## ---------------------------------------------------END OF TASK 4--------------------------------------------------------